<a href="https://colab.research.google.com/github/viam1-dev/colab_lineage_build/blob/lineage-15.1/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LineageOS Build

## Setting up SSH

In [ ]:
# SSH is currently prohibited from Google Colab (https://research.google.com/colaboratory/faq.html), BUT:
# - We need SSH to access logs properly since Colab doesn't show logs until the build "finishes"
# - We might need to measure speed 
# - We might locally debug and then push to GitHub
# SSH will be removed if Google fixes the issues with building LineageOS.

!pip install colab_ssh --upgrade
from colab_ssh import launch_ssh_cloudflared

# launch_ssh_cloudflared(password="<PUT_YOUR_PASSWORD_HERE>")
launch_ssh_cloudflared(password="lineage")

# SSH Config for Linux
# https://developers.cloudflare.com/cloudflare-one/connections/connect-apps/install-and-setup/tunnel-guide#1-download-and-install-cloudflared
# Create "~/.ssh/config"
# Copy Client machine configuration config bellow (appears after you run this block)
# Paste in SSH config file
# You can now SSH into Colab runtime via your shell of choice

# SSH Config for Windows PowerShell
# https://developers.cloudflare.com/cloudflare-one/connections/connect-apps/install-and-setup/tunnel-guide#1-download-and-install-cloudflared
# Create "C:\Users\USERNAME\.ssh\config"
# Copy Client machine configuration config bellow (appears after you run this block)
# Paste in SSH config file
# Download Cloudflare client and put in in C:\bin\cloudflared.exe and replace <PUT_THE_ABSOLUTE_CLOUDFLARE_PATH_HERE> with that.
# You can now SSH into Colab runtime via Windows PowerShell

## Installing build dependencies

In [ ]:
%%bash
sudo apt-get update
sudo apt-get install zip gcc-multilib g++-multilib \
    libc6-dev-i386 lib32ncurses5-dev lib32z1-dev imagemagick \
    libgl1-mesa-dev libxml2-utils xsltproc schedtool openjdk-8-jdk 
sudo ln -f -s /usr/bin/python2.7 /usr/bin/python

## Syncing LineageOS sources

In [ ]:
%%bash
mkdir ~/lineage
cd ~/lineage

mkdir ~/.bin
sudo curl http://commondatastorage.googleapis.com/git-repo-downloads/repo | sudo tee /usr/bin/repo > /dev/zero
sudo chmod 777 /usr/bin/repo
repo init --depth=1 -u https://github.com/LineageOS/android.git -b lineage-15.1
git clone --branch lineage-15.1 --single-branch --depth=1 https://github.com/viam1-dev/local_manifests.git .repo/local_manifests
repo sync -c --force-sync --optimized-fetch --no-tags --no-clone-bundle --prune -j$(nproc --all)
rm -rf .repo

## Building LineageOS

In [ ]:
%%bash
cd ~/lineage
source build/envsetup.sh
export ALLOW_MISSING_DEPENDENCIES=true
export LC_ALL=C
lunch lineage_viam1-user
make -j$(nproc --all)